In [ ]:
pip install pyproj shapely geopandas

### 구역별 영역 지도에 표시

In [ ]:
from pyproj import Transformer
import pyproj
import numpy as np
import pandas as pd
from shapely.geometry import Point as point
import geopandas as gpd
import folium
import random

import glob

target = 'path/to/*.shp'  # *.shp 상위 경로 디렉토리
# target 양식을 지키는 모든 파일 리스트형태로 저장
f_list = glob.glob(target)

def getMap(m,file_Path):
    # 구역별 색상선택을 위한 랜덤 변수
    color_random = "#" + "".join([random.choice("0123456789ABCDEF") for j in range(6)])
    # cp949로 안하면 인코딩이 안됨
    df = gpd.read_file(file_Path, encoding='cp949')
    # 중심좌표 구하고 4326 EPSG 좌표게 변환
    df['center_point'] = df['geometry'].centroid
    df['center_point'] = df['center_point'].to_crs(epsg=4326)
    # 경도
    df['lng'] = df['center_point'].map(lambda x:x.xy[0][0])
    # 위도
    df['lat'] = df['center_point'].map(lambda x:x.xy[1][0])
    # geometry의 중간 좌표 구하기
    df['center_point'] = df['geometry'].centroid
    # 경도 위도의 위치
    df_x = df['lng']
    df_y = df['lat']
    # 좌표들을 저장할 리스트
    coords = []
    # 좌표 추가
    for i in range(len(df)-1):
        x = df_x[i]
        y = df_y[i]
        coords.append([y, x])
    # 그리드 별 원 그리기
    for i in range(len(coords)):
        folium.Circle(
            location = coords[i],
            radius = 50,
            color = color_random,
            fill = 'crimson',
        ).add_to(m)
    # 다음 구역을 위한 좌표 초기화
    coords=[]
    # 지도 html에 저장하기
    m.save('C:\\Users\\Playdata\\Desktop\\study/map.html')
  
  # m = 지도 초기 좌표 지정
m = folium.Map([37.576454, 126.920860], zoom_start = 9)
for file in f_list:
    getMap(m,file)

In [ ]:
# 데이터 프레임에 주소 추가 

import glob
# 파일 경로 어디/서브디렉토리/파일.shp 형식을 찾아달라는 의미
target = r'C:/Users/Playdata/Desktop/study/mini project/**/*.shp'
# target 양식을 지키는 모든 파일을 리스트로 저장
f_list = glob.glob(target) # target 양식을 지키는 모든 파일을 리스트로 저장
# 예시
df = gpd.read_file(f_list[0],encoding='cp949')
df['center_point'] = df['geometry'].centroid
df['center_point'] = df['center_point'].to_crs(epsg=4326)
# 경도
df['lng'] = df['center_point'].map(lambda x:x.xy[0][0])
# 위도
df['lat'] = df['center_point'].map(lambda x:x.xy[1][0])
df['addr'] = f_list[0].split('\\')[-1].split('_')[1]

### 데이터 통합 코드

In [ ]:
import glob
import pandas as pd
# 파일 경로 어디/서브디렉토리/파일.shp 형식을 찾아달라는 의미
target = r'C:\py_source\mini project\mini project\유동인구분석/**/*.shp'
# target 양식을 지키는 모든 파일을 리스트로 저장
f_list = glob.glob(target) # target 양식을 지키는 모든 파일을 리스트로 저장
f_list_new =[]
for i in range(len(f_list)):
    df = gpd.read_file(f_list[i],encoding='cp949')
    df['center_point'] = df['geometry'].centroid
    df['center_point'] = df['center_point'].to_crs(epsg=4326)
    # 경도
    df['lng'] = df['center_point'].map(lambda x:x.xy[0][0])
    # 위도
    df['lat'] = df['center_point'].map(lambda x:x.xy[1][0])
    df['addr'] = f_list[i].split('\\')[-1].split('_')[1].replace('.shp', '')
    f_list_new.append(df)
    
result = pd.concat(f_list_new, axis=0)

# Hive query에서 쉼표를 인식 못하는 문제 발생
# polygen 출력값에서 쉼표 제거 후 ':' 으로 대체
result = result.astype({'geometry' : 'str'})
temp = pd.DataFrame({'new_geometry' : []})
# new_geometry라는 단일열을 갖고 문자열 형식의 geometry의 값을 갖는 새로운 데이터프레임
for item in result['geometry'] :
    tempp = pd.DataFrame({'new_geometry' : [item.replace(',', ':')]})
    temp = pd.concat([temp, tempp])
c_list = temp['new_geometry'].tolist()

# 기존 result 데이터 프레임에서 'geometry'열 제거 
result.drop('geometry',axis=1,inplace=True)

# ,를 :로 대체한 polygen을 원소로 갖는 리스트 c_list를 기존의 result에 새로운 열로 추가
result['geometry'] = c_list
result.columns
new_order = ['id', 'left', 'top', 'right', 'bottom', '_14age', '20_50T21_', '65age_',
       '20_60T9_18', 'weekend','geometry', 'center_point', 'lng', 'lat', 'addr']
result = result.reindex(columns=new_order)

result.to_csv('유동인구.csv',index=False,encoding='UTF-8')